In [1]:
# CREATION OF DATASET WHICH CONTAINS
# --------------------------------------------------------
#  ID_Citing | Category_Citing | Claims_Text                                      | ID_Cited      | References_Cited                                          | Content_Cited # Varying field depending on sample
# "3672233A1"  "X"               [{Claim 1: Content}, {Claim 2: Content}, ...]      "2709350A1"     '* abstract ** paragraphs [0065] - [0073] ** figure 6 *'    {pa01: Content, pnum: Content, fnum: Content}


In [2]:
import re
import json
import os
from html import unescape

# 04/03
# Task 2:
# Solve case "THE WHOLE DOCUMENT" (Extraction of only necessary info - PAGES + PARAGRAPHS + ABSTRACT + FIGURES + ¿LINES?): 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 14, 15 
# Count how many A,X,Y
# given 1000 pairs of documents (citing, cited)
# find the paragraphs of cited that were pointed out by the examiner
# and give the category, A or X

# Task 2:
                                                        #{
#(p20201, p20101) -> rank of cited paragraphs {
                                                # average
#(p20201, p20102) -> rank of cited paragraphs }
                                                            #average
#(p20202, p20103) -> rank of cited paragraphs
                                                        #}


# GOAL: Train & Test Dataset
# Take a look at the RAG DOC
# 5/04 PRESENTATION

# Pavlo and Pablo -
def cleaner_cited_references(cited_ID, paragraphs, json_cited):
    dict_content = {}
    dict_content2 = {}
    
    matching_entry = next(
        (element for element in json_cited if element["application_number"] + element["application_category"] == cited_ID), None
    )
    

    if matching_entry:
        fig_list = []

        # Cases:
        # Case 1. paragraphs = ""
        # Case 2. paragraphs = " the whole document"
        # Case 3. paragraphs = " the whole document * figure 3"
        #print("\n------------", matching_entry["application_number"], "-------------","\n")
        #print("Paragraphs:", paragraphs)
        if not paragraphs or 'the whole document' in paragraphs.lower():
            
            # DONE(except multiple cases)
            # Case 2 or 3
            
            if paragraphs:
                paragraphs = paragraphs.replace("* the whole document *", "")
                if not paragraphs:
                    return dict_content
                else:
                    
                    if "abstract" in paragraphs.lower() and "abstract" in matching_entry:
                        abstract_clean = re.sub(r'<[^>]+>', '', matching_entry["abstract"]).strip()[2:]
                        dict_content.update({"pa01":abstract_clean})
                    
                    # Extraction of Paragraphs IDs
                    paragraphs_IDs = extract_paragraph_numbers_the_whole_document(paragraphs)
                    # List of IDs not empty
                    #print("Paragraphs IDs - with whole doc:", paragraphs_IDs)
                    if paragraphs_IDs:
                        dict_paragraphs_content = extract_paragraphs_by_ids(matching_entry["description"], paragraphs_IDs)
                        #print("Paragraphs IDs Content - with whole doc:", dict_paragraphs_content)
                        dict_content.update(dict_paragraphs_content)
                        #dict_content2["Paragraphs_Text"] = list_paragraphs_content

                    # Extraction of Figure IDs

                    figure_IDs = extract_figure_ids_per_line(paragraphs)
                    #print("FIGURE IDS", figure_IDs)

                    figures_info = extraction_paragraphs_with_figures(matching_entry["description"], figure_IDs)
                    #print("FIGURES INFO", figures_info)
                    #print(figures_info)
                    dict_content.update(figures_info)
                    # Extraction of Figures IDs + Content

                    # figures_to_extract = extract_figure_ids_per_line(paragraphs)
                    # #print(figures_to_extract)
                    # content_all_figures = extract_lines_with_figure_references(matching_entry["description"])
                    #print(content_all_figures)



                    
                    # Extraction of specific Claim IDs + Content

                    # Extraction of Claims IDs - If not mentioned, returns empty list
                    claims_IDs_to_filter = extract_claims_from_paragraphs(paragraphs)
                    #print(claims_IDs_to_filter)
                    # If claims_IDs_to_filter == ["s"], return all claims
                    if claims_IDs_to_filter == [0]:
                        dict_content.update(extract_claim_info(json_cited, cited_ID))
                    if claims_IDs_to_filter:
                        # Extraction of content of Claims
                        dict_claims_content = extract_claim_info(json_cited, cited_ID, claims_IDs_to_filter)
                        # print(dict_claims_content)
                        # Matching specific IDs from dict_claims_content and claims_IDs_to_filter
                        # dict_claims_content_filtered = {key: dict_claims_content[key] for key in claims_IDs_to_filter}
                        #dict_claims_content_filtered = [claim for claim in dict_claims_content if any(key in claim for key in claims_IDs_to_filter)]
                        dict_content.update(dict_claims_content)


                    
                    return dict_content

                # # Extraction of complete Abstract
                # abstract_clean = re.sub(r'<[^>]+>', '', matching_entry["abstract"]).strip()
                # dict_content["pa01"] = abstract_clean
                
                # # Extraction of complete Description
                # description_clean = re.sub(r'<[^>]+>', '', matching_entry["description"]).strip()
                # dict_paragraphs_content_w = extract_paragraphs_by_ids(matching_entry["description"])
                # dict_content.update(dict_paragraphs_content_w)

                # # Extraction of all Claims content
                # dict_content.update(extract_claim_info(json_cited, cited_ID))
                # #dict_content["Claims"] = extract_claim_info(json_cited, cited_ID)

                # # Case 3

                # # Extraction of Specific Figure(s) within Complete                                                                     
                # figures_to_extract = extract_figure_ids_per_line(paragraphs)
                # #print(figures_to_extract)
                # content_all_figures = extract_lines_with_figure_references(matching_entry["description"])
                # #print(content_all_figures)
                # # We look for all references keywords = ["Figure", "Figures", "FIG.", "FIGS.", "Fig.", "Figs."] within the complete extracted content and search
                
                

                # return dict_content                                                                                                    
            
            #DONE
            # Case 1
            else: # We don't need to include all the info, leave empty don't skip the cited doc
                # Extraction of complete Abstract
                #abstract_clean = re.sub(r'<[^>]+>', '', matching_entry["abstract"]).strip()[3:]
                # We don't include it
                #dict_content["pa01"] = abstract_clean
                
                # Extraction of complete Description
                #description_clean = re.sub(r'<[^>]+>', '', matching_entry["description"]).strip()
                # We don't include it
                #dict_content["Description"] = description_clean

                # Extraction of all Claims content 
                # We don't include it
                #dict_content.update(extract_claim_info(json_cited, cited_ID))
                
                # Extraction of all Figure Description content
                # We don't include it - Not done
                #figures_to_extract = extract_figure_ids_per_line(paragraphs)
                #print(figures_to_extract)
                #content_all_figures = extract_lines_with_figure_references(matching_entry["description"])
                #print(content_all_figures)
                #dict_content["Figure Description"] = content_all_figures
                return dict_content

        else:
            # "* abstract ** paragraph [0016] - paragraph [0017] ** paragraph [0023] ** paragraph [0034] - paragraph [0044]; figure 3 *
            # "* claims 1-12; figures 1-4 *"
            


            # Extraction of complete Abstract
            #print("PARAGRAPHS", paragraphs)
            #print("MATCHING ENTRY", matching_entry)
            # DONE
            if "abstract" in paragraphs.lower() and "abstract" in matching_entry:
                #print(paragraphs)
                #print(matching_entry)
                #print(matching_entry["abstract"])
                abstract_clean = re.sub(r'<[^>]+>', '', matching_entry["abstract"]).strip()[2:]
                dict_content2.update({"pa01":abstract_clean})
            
            
            


            # Extraction of Paragraphs IDs + Content from Description
            
            # Extraction of Paragraphs IDs
            paragraphs_IDs = extract_and_expand_numbers(paragraphs)
            #print("Paragraphs IDs - WITHOUT whole doc:", paragraphs_IDs)
            #print("PARAGRAPHS IDs", paragraphs_IDs)
            # List of IDs not empty
            if paragraphs_IDs:
                dict_paragraphs_content = extract_paragraphs_by_ids(matching_entry["description"], paragraphs_IDs)
                #print("PARAGRAPHS INFO", dict_paragraphs_content)
                #print("Paragraphs IDs Content - WITHOUT whole doc:", dict_paragraphs_content)
                dict_content2.update(dict_paragraphs_content)
                #dict_content2["Paragraphs_Text"] = list_paragraphs_content


            # Extraction of Figure IDs

            figure_IDs = extract_figure_ids_per_line(paragraphs)
            #print("FiGURE IDs", figure_IDs)
            #print(figure_IDs)
            # List of IDs not empty
            #if figure_IDs:
                #list_figures_content =                                                                                                 # TO DO   
                #dict_content["Figures_Text"] = figure_IDs
            figures_info = extraction_paragraphs_with_figures(matching_entry["description"], figure_IDs)
            #print("FIGURE INFO", figures_info)
            #print(figures_info)
            dict_content2.update(figures_info)
            # Extraction of Figures IDs + Content

            # figures_to_extract = extract_figure_ids_per_line(paragraphs)
            # #print(figures_to_extract)
            # content_all_figures = extract_lines_with_figure_references(matching_entry["description"])
            #print(content_all_figures)



            
            # Extraction of specific Claim IDs + Content

            # Extraction of Claims IDs - If not mentioned, returns empty list
            claims_IDs_to_filter = extract_claims_from_paragraphs(paragraphs)
            #print(claims_IDs_to_filter)
            # If claims_IDs_to_filter == ["s"], return all claims
            if claims_IDs_to_filter == [0]:
                dict_content2.update(extract_claim_info(json_cited, cited_ID))
            if claims_IDs_to_filter:
                # Extraction of content of Claims
                dict_claims_content = extract_claim_info(json_cited, cited_ID, claims_IDs_to_filter)
                # print(dict_claims_content)
                # Matching specific IDs from dict_claims_content and claims_IDs_to_filter
                # dict_claims_content_filtered = {key: dict_claims_content[key] for key in claims_IDs_to_filter}
                #dict_claims_content_filtered = [claim for claim in dict_claims_content if any(key in claim for key in claims_IDs_to_filter)]
                dict_content2.update(dict_claims_content)


            
            return dict_content2

# Pavlo
def format_figure_references(text):
    # Regular expression pattern to match 'figures' followed by the references
    pattern = r'(figures?\s*[\d\w-]+(?:,\s*[\d\w-]+)*)'

    def replace_commas(match):
        # Replace commas followed by spaces with just a comma in the matched text
        match_text = re.sub(r',\s+', ',', match.group(0))
        # Correctly handle ranges and individual figures
        match_text = re.sub(r'(\d+)-(\d+|\w*)', lambda m: m.group(1) if not m.group(2) else m.group(0), match_text)
        match_text = re.sub(r'(\d+)-\d+-(\d+)', r'\1-\2', match_text)
        return match_text

    # Apply the replacement function to each match of the pattern in the text
    formatted_text = re.sub(pattern, replace_commas, text)
    return formatted_text

# Pavlo
def separate_alpha_numeric(text):
    numeric_part = ''.join(filter(str.isdigit, text))
    alphabetic_part = ''.join(filter(str.isalpha, text))
    return numeric_part, alphabetic_part

# Pavlo
def custom_sort_key(fig_id):
    try:
        numeric_part, alpha_part = separate_alpha_numeric(fig_id)
        numeric_value = int(numeric_part) if numeric_part else 0
        alpha_value = ord(alpha_part) if alpha_part else 0  # Sort by alphabetic part if present
        return numeric_value, alpha_value
    except Exception as e:
        return 0, 0

# Pavlo
def expand_range(start_fig, end_fig):
    start_num_part, start_alpha_part = separate_alpha_numeric(start_fig)
    end_num_part, end_alpha_part = separate_alpha_numeric(end_fig)

    if start_num_part == end_num_part and start_alpha_part and end_alpha_part:
        # Handle alphanumeric range, e.g., '3a' to '3c'
        start_index = ord(start_alpha_part)
        end_index = ord(end_alpha_part)
        return [f"{start_num_part}{chr(alpha)}" for alpha in range(start_index, end_index + 1)]
    elif not start_alpha_part and not end_alpha_part:
        # Handle numeric range, e.g., '1' to '3'
        return [str(i) for i in range(int(start_num_part), int(end_num_part) + 1)]
    else:
        # If the range is mixed or invalid, return an empty list
        return []

def extract_figure_ids_per_line(text):
    # First, format the figure references in the text
    formatted_text = format_figure_references(text)

    # Check if the text mentions "figures" without specifying any numbers
    if re.search(r'\bfigures?\b', formatted_text, re.IGNORECASE) and not re.search(r'\bfigures?\s+\d', formatted_text, re.IGNORECASE):
        return [0]

    # Then, extract all figure IDs from the formatted text
    figure_matches = re.findall(r'figures?\s*([0-9a-z,-]+(?:,\s*\d+(?:-\d+)?)*)', formatted_text, re.IGNORECASE)
    pattern = re.compile(r'\d+[A-Za-z]-\d+')
    figure_matches = [match for match in figure_matches if not pattern.search(match)]
    all_figure_ids = set()

    try:
        for match in figure_matches:
            figures_list = match.split(',')
            for fig in figures_list:
                fig = fig.strip()
                if '-' in fig:
                    start_fig, end_fig = fig.split('-')
                    all_figure_ids.update(expand_range(start_fig, end_fig))
                elif fig:
                    all_figure_ids.add(fig)
    except Exception as e:
        return sorted(all_figure_ids, key=custom_sort_key)

    return sorted(all_figure_ids, key=custom_sort_key)

# Pavlo and Pablo -
def extract_paragraph_numbers_the_whole_document(text):

    pattern_to_delete = re.compile(r'\bparagraph\s*\[\d+\]\s*-\s*paragraph\s*\[\d+\]')
    
    # Function to adjust the matched pattern
    def adjust_match(match):
        # Split the matched string by " - "
        parts = match.group().split(" - ")
        # Remove the second occurrence of "paragraph"
        adjusted = parts[0] + " - " + parts[1].replace("paragraph ", "", 1)
        return adjusted
    
    # Apply the adjustment to all matches in the text
    adjusted_text = re.sub(pattern_to_delete, adjust_match, text)

    # Normalize the text by replacing "and" with a comma and removing square brackets
    normalized_text = re.sub(r'\sand\s', ',', adjusted_text)
    normalized_text = re.sub(r'\s?-\s?', '-', normalized_text)
    normalized_text = re.sub(r'[\[\]]', '', normalized_text)

    # Define a regex pattern that matches paragraph references in various formats
    pattern = re.compile(r'\b(paragraphs|para\.|paragraph)\s*(\d+(?:-\d+)?(?:,\s*\d+(?:-\d+)?)*)', re.IGNORECASE)

    # Initialize an empty list to store all extracted paragraph numbers
    numbers = []

    # Find all matches and process each one
    for match in pattern.findall(normalized_text):
        parts = match[1].split(',')
        for part in parts:
            part = part.strip()
            if '-' in part:
                # Process ranges
                start, end = map(int, part.split('-'))
                numbers.extend(range(start, end + 1))
            else:
                # Process single numbers
                numbers.append(int(part))
    
    # Return a sorted list of unique paragraph numbers
    return sorted(set(numbers))

# Pablo --
def extract_and_expand_numbers(text):
    pattern = re.compile(r"\[(\d{4})\](?:\s*-\s*paragraph\s*\[(\d{4})\]|\s*-\s*\[(\d{4})\])?")
    matches = pattern.findall(text)
    numbers = []
    for match in matches:
        start = match[0]
        end = match[1] if match[1] else match[2]
        if start and end:
            start_num, end_num = int(start), int(end)
            numbers.extend(range(start_num, end_num + 1))
        elif start:
            numbers.append(int(start))
    return sorted(set(numbers))

def clean_text(html_text):
    clean_text = re.sub(r'<[^>]+>', '', text)

# Pablo--
def extract_paragraphs_by_ids(text, paragraph_ids=None):
    extracted_paragraphs = {}
    # Update to match any paragraph ID if None provided
    if paragraph_ids:
        for pid in paragraph_ids:
            pattern = re.compile(r'<p id="p{:04d}" num="\d+">(.*?)</p>'.format(int(pid)), re.DOTALL)
            match = pattern.search(text)
            if match:
                paragraph_text = match.group(1)
                paragraph_text = re.sub(r'<figref idref="[^"]+">(.*?)</figref>', r'\1', paragraph_text)
                paragraph_text = re.sub(r'<[^>]+>', '', paragraph_text)
                extracted_paragraphs[f"p{pid:04d}"] = paragraph_text
    else:
        paragraph_pattern = re.compile(r'<p id="p(\d+)"[^>]*>(.*?)</p>', re.DOTALL)
        matches = paragraph_pattern.findall(text)
        for pid, pcontent in matches:
            # Clean up <figref> tags and other HTML tags from paragraphs
            pcontent_clean = re.sub(r'<figref idref="[^"]+">(.*?)</figref>', r'\1', pcontent)
            pcontent_clean = re.sub(r'<[^>]+>', '', pcontent_clean).strip()
            extracted_paragraphs[f"p{int(pid):04d}"] = pcontent_clean
    return extracted_paragraphs

# Pablo--
def extract_claims_from_paragraphs(text):

    claim_numbers = []

    # Check if "claims" is mentioned without specific numbers
    if re.search(r'\bclaims?;|\bclaims?$', text, re.IGNORECASE):
        return [0]

    # Pattern to find claims and their ranges, correcting for spaces around dashes
    claims_pattern = re.compile(r'\bclaims?\s*([0-9,-]+)', re.IGNORECASE)
    text = re.sub(r'(\d+)\s*-\s*(\d+)', r'\1-\2', text)  # Remove spaces around dashes in ranges
    text = re.sub(r'\s*-\s*', '-', text)  # Remove spaces around dashes in ranges
    #print(text)
    claims_matches = claims_pattern.findall(text)
    
    for match in claims_matches:
        claim_list = match.split(',')
        for claim_range in claim_list:
            claim_range = claim_range.strip()
            if '-' in claim_range:
                parts = claim_range.split('-')
                # Ensure both parts are valid before proceeding
                if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                    start_claim, end_claim = map(int, parts)
                    claim_numbers.extend(range(start_claim, end_claim + 1))
            elif claim_range.isdigit():
                claim_numbers.append(int(claim_range))

    return sorted(set(claim_numbers))

# Pavlo & Pablo --
def extract_claim_info(json_citing, id_citing, cited_claims=None):
    matching_entry = next(
        (element for element in json_citing if element["application_number"] + element["application_category"] == id_citing), None
    )
    if matching_entry:
        claim_regex = r'(?:<clai)?m id="c-en-(\d+)" num="(\d+)">(.*?)</claim>'
        claim_text_regex = r'<claim-text>(.*?)</claim-text>'
        # Extracting all claims from the json_mapper
        claims = re.findall(claim_regex, matching_entry['claims'], re.DOTALL)
        claim_info = {}
        claim_list = []
        
        for claim_id, claim_num, claim_content in claims:
            claim_list.append(int(claim_num))
            claim_texts = re.findall(claim_text_regex, claim_content, re.DOTALL)
            claim_text = ' '.join(claim_texts).strip()
            claim_text = claim_text.replace("<claim-text>", " ")
            claim_text = claim_text.replace("</claim-text>", " ")
            claim_text = re.sub(r'<[^>]+>', '', claim_text).strip()
            claim_info[f"c-en-{int(claim_num):04d}"] = claim_text
            
            
        if cited_claims:
            return {f"c-en-{num:04d}": claim_info.get(f"c-en-{int(num):04d}", "Claim text not found") for num in cited_claims}
        else:
            return {f"c-en-{num:04d}": claim_info.get(f"c-en-{num:04d}", "Claim text not found") for num in claim_list}
    else:
        return {f"c-en-{num:04d}": "Claim text not found" for num in cited_claims}
        
# Pablo--
def extract_lines_with_figure_references(text):
    # Remove HTML tags but keep their content, especially for <figref>
    cleaned_text = re.sub(r'<figref idref=\"\w+\">', '', text)
    
    # Some cleaning
    cleaned_text = cleaned_text.replace('</figref>', '')
    
    # Lines split correctly
    well_split = split_text_with_complex_pattern(cleaned_text)
    
    # Keywords to filter
    keywords = ["figure", "figures", "Figure", "Figures", "FIG.", "FIGS.", "Fig.", "Figs."]
    
    figure_sentences = [line for line in well_split if any(keyword in line for keyword in keywords)]
    
    lista = []
    for text in figure_sentences:
        # Case 1: Removing <figref> including brackets and extracting the figure number or range directly.
        text = re.sub(r'\[\s*<figref idref="[^"]+">(FIGS?. [0-9a-zA-Z\(\)-]+)</figref>\s*\]', r'\1', text)
        
        # Case 2: Direct replacement of <figref> tags with their content.
        text = re.sub(r'<figref idref="[^"]+">(FIGS?. [0-9a-zA-Z\(\)-]+)</figref>', r'\1', text)

        # Case 3: Similar to case 2 but specifically targeting the pattern with brackets at the start.
        text = re.sub(r'\[\s*<figref idref="[^"]+">(Fig. [0-9]+)\s*\]?\s*\1</figref>', r'\1', text)

        # Clean remaining HTML tags but keep their content
        text = re.sub(r'<[^>]+>', '', text)
        lista.append(text)

    return lista

# Pavlo
def extraction_paragraphs_with_figures(text, figure_list):
    paragraph_pattern = re.compile(r'<p id="p(\d+)"[^>]*>(.*?)</p>', re.DOTALL)
    figref_pattern = re.compile(r'<figref idref="f(\d+)[A-Za-z]*">(.*?)</figref>', re.DOTALL)
    paragraphs_with_figures = {}

    if figure_list == [0]:
        # If figure_list is [0], include paragraphs with any figure reference
        for pnum, ptext in paragraph_pattern.findall(text):
            if figref_pattern.search(ptext):  # Check if paragraph contains any figure reference
                clean_text = re.sub(r'<[^>]+>', '', ptext).strip()
                formatted_text = format_figure_references(clean_text)
                paragraphs_with_figures["p" + pnum] = formatted_text
    else:
        # Otherwise, include paragraphs with specific figures from figure_list
        for pnum, ptext in paragraph_pattern.findall(text):
            fig_refs = figref_pattern.findall(ptext)
            for fig_id, fig_text in fig_refs:
                if any(fig in fig_text for fig in figure_list):
                    clean_text = re.sub(r'<[^>]+>', '', ptext).strip()
                    formatted_text = format_figure_references(clean_text)
                    paragraphs_with_figures["p" + pnum] = formatted_text
                    break  # Include paragraph once a matching figure is found

    return paragraphs_with_figures

# Pablo--
def split_text_with_complex_pattern(text):
    """
    Splits the given text into lines at periods, handling complex figure references and enumerations without splitting them,
    and ensuring correct sentence boundaries are recognized.
    """
    pattern = re.compile(
    r'\.(?!\s*(Fig|Figs|FIG|FIGS)\.\s*\d+([-,]\s*\d+)*\s+|<\/?\s*(li|ul|p|!--)>)\s*(?=[A-Z])'
    )
    
    # Split the text using the defined pattern
    lines = pattern.split(text)
    #print("Lines:", lines)
    # Ensure that we filter out any None before processing
    lines = [line for line in lines if line is not None]

    return lines

In [3]:
# Pavlo and Pablo
def process_data(json_mapper, json_citing, json_cited):
    # List to store processed data
    data = []
    
    # We count total citing patents
    counter_total = 0
    
    # We count the cases 2 or 3
    counter_whole_variations = 0
    counter_A_non_empty = 0
    counter_Y_non_empty = 0
    counter_X_non_empty = 0
    counter_paragraphs_empty = 0
    counter_A_empty = 0
    counter_Y_empty = 0
    counter_X_empty = 0
    counter_empty = 0
    
    # Processing each dictionary in "MAPPING_DATASET_citing_to_citations.json"
    for key, value in json_mapper.items():
        for dict_cited in value:
            if dict_cited["type"] in ["X", "Y", "A"]:
                if not dict_cited["claims"]:
                    continue
                else:
                    quadruple_dict = {
                        "ID_Citing": key,
                        "Category_Cited": dict_cited["type"],
                        "Claims_Text": extract_claim_info(json_citing, key, dict_cited["claims"]),
                        "ID_Cited": dict_cited["cited_id"],
                        "References_Cited" : dict_cited["paragraphs"],
                        "Content_Cited": cleaner_cited_references(dict_cited["cited_id"], dict_cited["paragraphs"], json_cited)
                    }
                    data.append(quadruple_dict)
                    
                    if quadruple_dict["Category_Cited"] == "A" and quadruple_dict["Content_Cited"]:
                        counter_A_non_empty +=1
                    elif quadruple_dict["Category_Cited"] == "A" and not quadruple_dict["Content_Cited"]:
                        counter_A_empty +=1
                    
                    if quadruple_dict["Category_Cited"] == "X" and quadruple_dict["Content_Cited"]:
                        counter_X_non_empty +=1
                    elif quadruple_dict["Category_Cited"] == "X" and not quadruple_dict["Content_Cited"]:
                        counter_X_empty +=1
                    
                    if quadruple_dict["Category_Cited"] == "Y" and quadruple_dict["Content_Cited"]:
                        counter_Y_non_empty +=1
                    elif quadruple_dict["Category_Cited"] == "Y" and not quadruple_dict["Content_Cited"]:
                        counter_Y_empty +=1
                    
                    if not quadruple_dict["References_Cited"]:
                        counter_paragraphs_empty += 1
                    if not quadruple_dict["Content_Cited"]:
                        counter_empty +=1
                    else:
                        counter_total +=1   
                           
    print("Nº A non-empty:", counter_A_non_empty)
    print("Nº Y non-empty:", counter_Y_non_empty)
    print("Nº X non-empty:", counter_X_non_empty)
    print("Nº Paragraphs empty", counter_paragraphs_empty)
    print("Nº A empty:", counter_A_empty)
    print("Nº Y empty:", counter_Y_empty)
    print("Nº X empty:", counter_X_empty)
    
    print("Total A + X + Y ~:", counter_A_non_empty + counter_X_non_empty + counter_Y_non_empty)
    print("Total references empty:", counter_empty)
    print("Total references non-empty :", counter_total)
    return data

# Pavlo and Pablo
def open_json_file(path_to_json):
    # Opening the JSON file
    with open(path_to_json, 'r') as file:
        # Loading JSON
        json_dict = json.load(file)
    return json_dict

# Pavlo and Pablo
def create_json(data, path):
    # Creation of Directory if doesn't exist
    directory = '/bigstorage/Pablo_TER'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # File Path to store data information
    file_path = os.path.join(directory, 'Q7.json')

    # Writing Q7 Information on JSON
    with open(file_path, 'w') as file:
        # Converting from list of dictionaries to JSON + indent=4 for readability
        json.dump(data, file, indent=4)

# Pavlo and Pablo
def main():
    path_to_json_mapper = "/bigstorage/ita/clean/private/MAPPING_DATASET_citing_to_citation.json"
    path_to_json = "/bigstorage/ita/clean/private/CONTENT_DATASET_citing_patents_2020.json"
    path_to_json_content_cited = "/bigstorage/ita/clean/private/CONTENT_DATASET_cited_patents_by_2020.json"
    #test_paragraphs_path = "/bigstorage/Pablo_TER/EP_2020_attempts/EP_2020.json"
    path_to_Q7 ='/bigstorage/Pablo_TER'
    json_citing = open_json_file(path_to_json)
    json_cited = open_json_file(path_to_json_content_cited)
    json_mapper = open_json_file(path_to_json_mapper)
    data = process_data(json_mapper, json_citing, json_cited)
    create_json(data, path_to_Q7)

if __name__ == "__main__":
    main()

Nº A non-empty: 6189
Nº Y non-empty: 2232
Nº X non-empty: 3671
Nº Paragraphs empty 2631
Nº A empty: 1058
Nº Y empty: 767
Nº X empty: 2357
Total A + X + Y ~: 12092
Total references empty: 4182
Total references non-empty : 12092


# Information related to Q7.JSON

- MAPPING_DATASET_citing_to_citations.json -> 20K

- Total Cited Patents in 2020: 16274 ( / 20k <- MAPPING_DATASET_citing_to_citations.json) 

- Total Cited Patents in 2020 with non-empty paragraphs: 12092 (~ 74,30% )

- Total Cited Patents in 2020 with empty paragraphs: 4182 (~ 25,7% )

- The whole Document:

    - "The Whole Document" is present in 1752 Cited References

    - "The Whole Document" + something" in 87 Cited References (-> in our algorithm we consider 68 / 87 from them)

    - "The Whole Document" only is in 1588 Cited References

- Total Cited Patents in 2020 with empty paragraphs ( paragraphs = "" (3079) + paragraphs = "the whole document" (1588) + paragraphs = "figure 1a-4b" or paragraphs = "3a-VII" (1998) ): 6665 (~ 35,62 % )

- Total Time Execution for Cited patents in 2020: 2m 21.1s

# General Information

- MAPPING_DATASET_citing_to_citations.json -> 20K

- CONTENT_DATASET_CITED_PATENTS_BY_2020.JSON -> 12.219 ~ 12K

- CONTENT_DATASET_CITING_PATENTS_2020.JSON -> 12.093 ~12K

- CONTENT_DATASET_DISTINCT_UNCITED_PATENTS.JSON -> 10K

- CONTENT_DATASET_UNCITED_PATENTS.JSON -> 10K